In [2]:
import sys
sys.path += ['../../..']
sys.path += ['../..']
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import *
from tqdm import tqdm
import datetime
import tensorflow as tf
import tensorflow.keras as keras



from data_preparation import (
    Session, match_time, data_prep, sample_prep, stack_samples
)
%load_ext autoreload
%autoreload 2


# 1. Get data

In [7]:
!sftp -r kawa@192.168.0.28:projects/car1/src/training/9Jul/log ../data/new_data

ssh: connect to host 192.168.0.28 port 22: No route to host
Connection closed.  
Connection closed


# 2. Load data

In [3]:
sessions = Session.load_multiple_session('../../data/10Jul')



[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5f2c4bda7780] moov atom not found
2024-07-11 18:56:33.433550: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-11 18:56:33.465758: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-11 18:56:33.466003: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvald

In [46]:
[data_prep(s) for s in tqdm(sessions[1:2])]; 
[sample_prep(s) for s in tqdm(sessions[1:2])]; 

x, y = stack_samples(sessions[1:2])


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 52.92it/s]


In [45]:
sessions[1:2]

[Session('../../data/10Jul/model_validation2024-07-10 21:27:07.316460')]

In [5]:
model = keras.models.load_model("./model4-10Jul.keras") 


In [75]:
tf.saved_model.save(model, "saved_model_keras_dir")

AssertionError: Tried to export a function which references an 'untracked' resource. TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly. See the information below:
	Function name = b'__inference_signature_wrapper_serving_default_6090'
	Captured Tensor = <ResourceHandle(name="seed_generator/seed_generator_state/0", device="/job:localhost/replica:0/task:0/device:GPU:0", container="Anonymous", type="tensorflow::Var", dtype and shapes : "[ DType enum: 22, Shape: [2] ]")>
	Trackable referencing this tensor = <tf.Variable 'seed_generator/seed_generator_state:0' shape=(2,) dtype=uint32>
	Internal Tensor = Tensor("6050:0", shape=(), dtype=resource)

In [74]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

AttributeError: 'Sequential' object has no attribute '_get_save_spec'

In [71]:
y_pred = model.predict(x, batch_size=64)
y_pred2 = model.predict(x, batch_size=64)

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [69]:
px.line(np.c_[y[:, 2], y_pred[:, 2]]*100, title='val')

In [53]:
idxs = sessions[1].data['camera_df'].query('mean_speed>0').index#

cdf = sessions[1].data['camera_df'].copy()
cdf.loc[idxs, ['y_pred0', 'y_pred1', 'y_pred2','y_pred3']] = y_pred*100


In [70]:
px.line(cdf, x='time', y=['angular_velocity_smooth_200', 'y_pred3'])

In [60]:
mdf = sessions[1].data['FrameMLToAngularVelocity']#['out']/100
mdf['time'] = mdf['run_model']

In [67]:
px.line(pd.concat([mdf, cdf]), x='time', y=['out',  'y_pred2'])

In [73]:
px.line(pd.concat([mdf, cdf]), x='time', y=['out', 'angular_velocity_smooth_100', 'y_pred1'])

In [36]:
px.histogram(1/sessions[1].data['FrameMLToAngularVelocity']['timelapsed'])

In [17]:
sessions[0].data['camera_df']


px.line(sessions[1].data['camera_df'], x ='time', y='angular_velocity_smooth_200')

In [18]:
px.line(sessions[1].data['FrameMLToAngularVelocity'], x ='run_model', y='out')